In [13]:
import os
import sys
import random
import gzip
import os
import numpy as np
from scipy import spatial

## paths to files. Do not change this
simInputFile = "Q1/word-similarity-dataset"
analogyInputFile = "Q1/word-analogy-dataset"
#analogyInputFile = "analogy_small.txt"
vectorgzipFile = "Q1/glove.6B.300d.txt.gz"
#vectorTxtFile = "Q1/glove.6B.300d.txt"   # If you extract and use the gz file, use this.
analogyTrainPath = "Q1/wordRep/"
simOutputFile = "Q1/simOutput.csv"
simSummaryFile = "Q1/simSummary.csv"
anaSOln = "Q1/analogySolution.csv"
Q4List = "Q4/wordList.csv"

random.seed(100)

In [14]:
# Similarity Dataset
simDataset = [item.split(" | ") for item in open(simInputFile).read().splitlines()]
# Analogy dataset
analogyDataset = [[stuff.strip() for stuff in item.strip('\n').split('\n')] for item in open(analogyInputFile).read().split('\n\n')]

def vectorExtract(simD = simDataset, anaD = analogyDataset, vect = vectorgzipFile):
    simList = [stuff for item in simD for stuff in item]
    analogyList = [thing for item in anaD for stuff in item[0:4] for thing in stuff.split()]
    #simList.extend(analogyList)
    wordList = set(simList)
    print(len(wordList))
    wordDict = dict()
    
    vectorFile = gzip.open(vect, 'r')
    for line in vectorFile:
        if line.split()[0].strip().decode("utf-8") in wordList:
            wordDict[line.split()[0].strip().decode('utf-8')] = line.split()[1:]
    
    
    vectorFile.close()
    print('retrieved', len(wordDict.keys()))
    return wordDict, simList

# Extracting Vectors from Analogy and Similarity Dataset


In [15]:
worddict, simlist = vectorExtract()

170
retrieved 166


In [16]:
#Get data from similarity file
def get_sim_data(sim = simInputFile):
    fin = open(sim, 'r')
    qs = []
    for line in fin:
        q = [item.strip() for item in line.split('|')]
        qs.append(q)
    return qs

#Check if all the words in the question exist in word dictionary
def indict(q, worddict):
    for w in q:
        if(w not in worddict):
            return False
    return True
    
        

In [17]:
#Find cosine, euclidean and manhattan similarity. We are using scipy package for distance calculation

def find_sim(sim = simInputFile, worddict = worddict):
    qs = get_sim_data()
    for q in qs:
        if(not indict(q, worddict)):
            qs.remove(q)
    #print(qs, len(qs))
    vec = [[(worddict[word]) for word in question ] for question in qs]
    vec = [[[float(el) for el in vector] for vector in question] for question in vec]
    cos = []
    euclidean = []
    manhattan = []
    for q in vec:
        dotp = []
        dist1 = []
        dist2 = []
        q = np.array(q)
        for i in range(1,5):
            dotp.append(spatial.distance.cosine(q[0], q[i])) 
            dist1.append(spatial.distance.euclidean(q[0], q[i]))
            dist2.append(spatial.distance.cityblock(q[0], q[i]))
        dotp = np.array(dotp)
        dist1 = np.array(dist1)
        dist2 = np.array(dist2)
        cos.append(np.argmax(dotp))
        euclidean.append(np.argmax(dist1))
        manhattan.append(np.argmax(dist2))
    #vec = np.array(vec)
    c1 = 0
    c2 = 0
    c3 = 0
    for answer in cos:
        if(answer==0):
            c1 = c1 + 1
    f = open("output/simOutput.txt", 'w')
    print("Cosine similarity: ", c1*100.0/len(cos),"% accuracy")
    f.write("Cosine similarity: "+ str(c1*100.0/len(cos))+"% accuracy")
    
    for answer in euclidean:
        if(answer==0):
            c2 = c2 + 1
    print("Euclidean similarity: ", c2*100.0/len(cos),"% accuracy")
    
    f.write("Cosine similarity: "+ str(c2*100.0/len(cos))+"% accuracy")
    
    for answer in manhattan:
        if(answer==0):
            c3 = c3 + 1
            
    print("Manhattan similarity: ", c3*100.0/len(cos),"% accuracy")
    f.write("Cosine similarity: "+ str(c3*100.0/len(cos))+"% accuracy")
    
    #print(cos,'\n', euclidean, '\n', manhattan)
find_sim()
#get_sim_data()

Cosine similarity:  10.81081081081081 % accuracy
Euclidean similarity:  13.513513513513514 % accuracy
Manhattan similarity:  13.513513513513514 % accuracy
